<a href="https://colab.research.google.com/github/AlgorithmBurv/RAG-LLM-for-Contextual-Answering/blob/main/RAG_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai langchain langchain-google-genai langchain_community pypdf chromadb

# Libararies

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import NLTKTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from IPython.display import Markdown as md
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Load Teks

In [ ]:
!curl -o /content/Faktor_Risiko_Dominan_Penyakit_Jantung_Koroner_di_Indonesia.pdf "https://d1wqtxts1xzle7.cloudfront.net/99914958/59f6a3929ef42f8dc30e6d8d957f6321b56a-libre.pdf?1678951855=&response-content-disposition=inline%3B+filename%3DFaktor_Risiko_Dominan_Penyakit_Jantung_K.pdf&Expires=1753448446&Signature=Vypd6m5BCvgk0lJwNI0FaEpyNFZhK3z0bvSBJJl6S~pj2SYbwRHCGTzMTZhA-zk1SM6sntxgsgvL5eCGakINmmTASuj236u2Xv--hZN-Qeq8FhxXSvkZgVf6PmpHfn5CSCg4dU8AIkTiEK55tQsRQJQrOvIwdnM3QwvT6xAgvnrx3LS04-CUTOEAsHOh3cvPHlyjQSbJ8JKPu9lMVS0XyYhds~39dEYVHTlBvcuy-JmCZHn8wcopUDUK7PbpVmRp1MWVFH5Te0QwvzqWsZb47~8o88tH19FW8tXQcZdSXNe12zmM044sJYKzAMapy7k3XoBWRLhqk9hcY-4SmmAkmA__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  220k  100  220k    0     0  1448k      0 --:--:-- --:--:-- --:--:-- 1450k


In [ ]:
loader = PyPDFLoader("/content/Faktor_Risiko_Dominan_Penyakit_Jantung_Koroner_di_Indonesia.pdf")
pages = loader.load_and_split()

In [ ]:
md(pages[0].page_content)

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
153
Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia
DOMINANT RISK FACTORS OF CORONARY HEART DISEASE IN INDONESIA
Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani
  Pusat Penelitian dan Pengembangan Sumber Daya dan Pelayanan Kesehatan
Jl. Percetakan Negara No. 29 Jakarta 10560, Indonesia
E - mail: lannywatighani@yahoo.com
        Submitted : 5-9-2016,    Revised : 13-9-2016,    Revised : 14-9-2016,    Accepted : 23-9-2016
Abstract
Coronary heart disease is still a major cause of mortality and morbidity with great socio-economic 
impact like stroke. Coronary heart disease is preventable by early detection and risk factors control. 
This study aimed to identify dominant risk factors of coronary heart disease in Indonesia using 
National Basic Health Survey 2013 data. Data were analysed using SPSS 16 with complex sample. A 
total of 722.329 respondent age ≥15 years old, consisting of 347.823 male and 374.506 female were 
analyzed. Prevalence of coronary heart disease was 1.5 % (95% CI 1.4–1.5). Several factors were 
identified as risk factors of coronary heart disease such as hypertension, mental disorder, diabetes 
mellitus, stroke, age ≥ 40 years old, smoking, female, low level of education, central obesity, and low 
level of social economic status with adjusted odds ratio range from 1.30 to 10.09. The dominant risk 
factors were hypertension, mental disorder, and diabetes mellitus. Promotion of healthy lifestyle and 
early detection since early age need to be enhanced to minimize the risk factors as well as coronary 
heart disease.
Key words : coronary heart disease, risk factor, basic health survey 2013.
Abstrak
Penyakit Jantung Koroner (PJK) masih merupakan penyebab kematian dan kesakitan utama yang 
berdampak secara sosioekonomi seperti juga stroke. Penyakit ini dapat dicegah dengan deteksi dini 
dan pengendalian faktor risiko. Penelitian ini bertujuan untuk mendapatkan faktor risiko dominan 
penderita jantung koroner di Indonesia dengan  menggunakan data sekunder dari Survei Riset 
Kesehatan Dasar Indonesia tahun 2013. Analisis data menggunakan SPSS 16 dengan c omplex 
sample.  Dari 722.329 responden usia ≥ 15 tahun didapat prevalensi penyakit jantung koroner sebesar 
1,5 % (95 % CI 1,4–1,5). Hipertensi, gangguan mental emosional, diabetes melitus, stroke, usia ≥ 
40 tahun, kebiasaan merokok, jenis kelamin perempuan, tingkat pendidikan rendah, obesitas sentral, 
dan status sosial ekonomi rendah merupakan faktor risiko PJK dengan rasio odd berkisar dari 1,30 
hingga 10,09. Faktor risiko dominan penyakit jantung koroner adalah hipertensi, gangguan mental 
emosional, dan diabetes melitus. Upaya promotif dan deteksi dini faktor risiko sejak usia dini perlu 
ditingkatkan untuk memperkecil kejadian faktor risiko maupun penyakit jantung koroner.
Kata kunci : penyakit jantung koroner, faktor risiko, Riskesdas 2013.

# Model LLM


In [ ]:
api_key = userdata.get('API_KEY')
chat_model = ChatGoogleGenerativeAI(google_api_key=api_key,
                                    model='gemini-2.5-flash',
                                    temperature=0.9)

# Chunking

In [ ]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)


chunks= text_splitter.split_documents(pages)
print(len(chunks))

53


In [ ]:
chunks[0].page_content

'Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )\n153\nFaktor Risiko Dominan Penyakit Jantung Koroner di Indonesia\nDOMINANT RISK FACTORS OF CORONARY HEART DISEASE IN INDONESIA\nLannywati Ghani, Made Dewi Susilawati dan Harli Novriani\n  Pusat Penelitian dan Pengembangan Sumber Daya dan Pelayanan Kesehatan\nJl.\n\nPercetakan Negara No.\n\n29 Jakarta 10560, Indonesia\nE - mail: lannywatighani@yahoo.com\n        Submitted : 5-9-2016,    Revised : 13-9-2016,    Revised : 14-9-2016,    Accepted : 23-9-2016\nAbstract\nCoronary heart disease is still a major cause of mortality and morbidity with great socio-economic \nimpact like stroke.\n\nCoronary heart disease is preventable by early detection and risk factors control.\n\nThis study aimed to identify dominant risk factors of coronary heart disease in Indonesia using \nNational Basic Health Survey 2013 data.\n\nData were analysed using SPSS 16 with complex sample.'

# Embedding

In [ ]:
embedding_model= GoogleGenerativeAIEmbeddings(google_api_key=api_key,model="models/embedding-001")

In [ ]:
embedding_model

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c14635d2bd0>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

# Vector Databases

In [ ]:
db = Chroma.from_documents(chunks, embedding_model, persist_directory='./chroma_db_')
db.persist()

In [ ]:
db_connection = Chroma(persist_directory='./chroma_db_', embedding_function=embedding_model)

In [ ]:
retriever= db_connection.as_retriever(search_kwargs={'k':50})
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [ ]:
check_response = retriever.invoke('jelaskan apa saja faktor risiko dominan untuk penyakit jantung koroner di Indonesia?')
md(check_response[0].page_content)

Berbagai 
penelitian telah berhasil mengidentifikasi faktor-
faktor risiko penyakit jantung koroner  antara lain 
herediter, usia, jenis kelamin, sosioekonomi, letak  
geografi, makanan tinggi lemak dan kalori, kurang 
makan sayur buah, merokok, alkohol, aktifitas 
fisik kurang, hipertensi, obesitas, diabetes mellitus, 
aterosklerosis, penyakit arteri perifer, stroke dan 
dislipidemia.8-11 Faktor risiko dominan penyakit 
jantung koroner secara nasional belum ada.

# Prompt

In [ ]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content='Anda adalah AI pintar yang dapat menjawab pertanyaan sesuai konteks yang diberikan terkait penyakit jantung koroner di Indonesia.'),
    HumanMessagePromptTemplate.from_template('''jawab pertanyaan berikut berdasarkan konteks.
konteks: {context}
pertanyaan: {question}
jawaban: '''),
])


In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser= StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

formatted_docs= format_docs(chunks)
md(formatted_docs)

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
153
Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia
DOMINANT RISK FACTORS OF CORONARY HEART DISEASE IN INDONESIA
Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani
  Pusat Penelitian dan Pengembangan Sumber Daya dan Pelayanan Kesehatan
Jl.

Percetakan Negara No.

29 Jakarta 10560, Indonesia
E - mail: lannywatighani@yahoo.com
        Submitted : 5-9-2016,    Revised : 13-9-2016,    Revised : 14-9-2016,    Accepted : 23-9-2016
Abstract
Coronary heart disease is still a major cause of mortality and morbidity with great socio-economic 
impact like stroke.

Coronary heart disease is preventable by early detection and risk factors control.

This study aimed to identify dominant risk factors of coronary heart disease in Indonesia using 
National Basic Health Survey 2013 data.

Data were analysed using SPSS 16 with complex sample.

Data were analysed using SPSS 16 with complex sample.

A 
total of 722.329 respondent age ≥15 years old, consisting of 347.823 male and 374.506 female were 
analyzed.

Prevalence of coronary heart disease was 1.5 % (95% CI 1.4–1.5).

Several factors were 
identified as risk factors of coronary heart disease such as hypertension, mental disorder, diabetes 
mellitus, stroke, age ≥ 40 years old, smoking, female, low level of education, central obesity, and low 
level of social economic status with adjusted odds ratio range from 1.30 to 10.09.

The dominant risk 
factors were hypertension, mental disorder, and diabetes mellitus.

Promotion of healthy lifestyle and 
early detection since early age need to be enhanced to minimize the risk factors as well as coronary 
heart disease.

Key words : coronary heart disease, risk factor, basic health survey 2013.

Key words : coronary heart disease, risk factor, basic health survey 2013.

Abstrak
Penyakit Jantung Koroner (PJK) masih merupakan penyebab kematian dan kesakitan utama yang 
berdampak secara sosioekonomi seperti juga stroke.

Penyakit ini dapat dicegah dengan deteksi dini 
dan pengendalian faktor risiko.

Penelitian ini bertujuan untuk mendapatkan faktor risiko dominan 
penderita jantung koroner di Indonesia dengan  menggunakan data sekunder dari Survei Riset 
Kesehatan Dasar Indonesia tahun 2013.

Analisis data menggunakan SPSS 16 dengan c omplex 
sample.

Dari 722.329 responden usia ≥ 15 tahun didapat prevalensi penyakit jantung koroner sebesar 
1,5 % (95 % CI 1,4–1,5).

Hipertensi, gangguan mental emosional, diabetes melitus, stroke, usia ≥ 
40 tahun, kebiasaan merokok, jenis kelamin perempuan, tingkat pendidikan rendah, obesitas sentral, 
dan status sosial ekonomi rendah merupakan faktor risiko PJK dengan rasio odd berkisar dari 1,30 
hingga 10,09.

Faktor risiko dominan penyakit jantung koroner adalah hipertensi, gangguan mental 
emosional, dan diabetes melitus.

Upaya promotif dan deteksi dini faktor risiko sejak usia dini perlu 
ditingkatkan untuk memperkecil kejadian faktor risiko maupun penyakit jantung koroner.

Kata kunci : penyakit jantung koroner, faktor risiko, Riskesdas 2013.

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
154
PENDAHULUAN 
 Penyakit jantung koroner (PJK) adalah 
penyakit yang disebabkan adanya plak yang 
menumpuk di dalam arteri koroner yang mensuplai 
oksigen ke otot jantung.

1 Penyakit ini termasuk 
bagian dari penyakit kardiovaskuler yang paling 
umum terjadi.

Penyakit kardiovaskuler merupakan 
gangguan dari jantung dan pembuluh darah 
termasuk stroke,  penyakit jantung rematik dan 
kondisi lainnya (WHO).2  
 Menurut statistik dunia, ada 9,4 juta 
kematian setiap tahun yang disebabkan oleh 
penyakit kardiovaskuler dan 45% kematian 
tersebut disebabkan oleh penyakit jantung koroner.

Diperkirakan angka tersebut akan meningkat 
hingga 23,3 juta pada tahun 2030.3,4  
 Di Indonesia salah satu penyakit 
kardiovaskular yang terus menerus menempati 
urutan pertama adalah penyakit jantung koroner.

5 
Menurut survei Sample Registration System angka 
kematian penyakit jantung koroner 12,9% dari 
seluruh kematian.

6 Prevalensi penyakit jantung 
koroner berdasarkan diagnosis dokter yang 
dilakukan Riset Kesehatan Dasar (Riskesdas) 2013 
sebesar 0,5% sedangkan berdasarkan diagnosis 
dokter atau gejala sebesar 1,5%.

Hasil Riskesdas ini 
menunjukkan penyakit jantung koroner berada pada 
posisi ketujuh tertinggi Penyakit Tidak Menular 
(PTM) di Indonesia.7
 Penyakit jantung koroner (PJK) ini masih 
merupakan masalah kesehatan yang penting dan 
berdampak secara sosio-ekonomi karena biaya 
obat-obatan yang cukup mahal, lamanya waktu 
perawatan dan pengobatan, serta pemeriksaan 
penunjang lain yang diperlukan dalam proses 
pengobatan.

Upaya pencegahan melalui deteksi 
dini faktor risiko dan upaya pengendaliannya sangat 
penting dilakukan.

Identifikasi faktor risiko Penyakit 
jantung koroner (PJK) sangat bermanfaat untuk 
perencanaan intervensi pencegahan.

Berbagai 
penelitian telah berhasil mengidentifikasi faktor-
faktor risiko penyakit jantung koroner  antara lain 
herediter, usia, jenis kelamin, sosioekonomi, letak  
geografi, makanan tinggi lemak dan kalori, kurang 
makan sayur buah, merokok, alkohol, aktifitas 
fisik kurang, hipertensi, obesitas, diabetes mellitus, 
aterosklerosis, penyakit arteri perifer, stroke dan 
dislipidemia.8-11 Faktor risiko dominan penyakit 
jantung koroner secara nasional belum ada.

Analisis ini bertujuan untuk mendapatkan 
faktor risiko dominan penyakit jantung koroner 
di Indonesia.

Diharapkan hasil analisis ini dapat 
bermanfaat sebagai bahan pertimbangan dalam 
upaya intervensi pencegahan melalui deteksi dini 
dan promosi kesehatan secara umum.

BAHAN DAN METODE
 Penelitian ini merupakan analisis lanjut 
dari Riskesdas 2013, dimana data yang dianalisis 
adalah sampel berusia 15 tahun atau lebih.

Data  
dikumpulkan dengan metode wawancara oleh 
enumerator yang telah dilatih.

Data  
dikumpulkan dengan metode wawancara oleh 
enumerator yang telah dilatih.

Variabel yang 
dianalisis yaitu usia, jenis kelamin, sosioekonomi, 
demografi, dan gaya hidup seperti merokok, kurang 
konsumsi sayur buah, kurang aktifitas fisik.

Selain 
itu, ditanyakan pula apakah responden pernah 
didiagnosis penyakit jantung koroner oleh tenaga 
kesehatan dan apakah menderita gejala penyakit 
jantung koroner serta penyakit tidak menular 
lainnya.

Pemeriksaan fisik meliputi pengukuran 
antropometri dan tekanan darah.

Disain penelitian 
adalah potong lintang ( cross sectional ) sesuai 
metode Riskesdas 2013.7,11
 Sampel adalah data seluruh anggota rumah 
tangga (ART) berusia ≥ 15 tahun.

Data yang tidak 
lengkap dieksklusi.

Analisis  dilakukan pada sampel 
sejumlah 722.329 anggota rumah tangga terdiri dari 
347.823 laki-laki dan 374.506 perempuan.

Data karakteristik sosiodemografi, perilaku, 
hasil pengukuran, dan gambaran klinis presponden 
Penyakit jantung koroner (PJK) dianalisis 
menggunakan complex sample  secara deskriptif, 
dilanjutkan dengan uji bivariat dan multivariat 
(analisis regresi logistik) untuk mendapatkan faktor 
risiko dominan.

Data mentah ini pernah dianalisis untuk 
mencari faktor risiko dominan penderita stroke

(analisis regresi logistik) untuk mendapatkan faktor 
risiko dominan.

Data mentah ini pernah dianalisis untuk 
mencari faktor risiko dominan penderita stroke 
di Indonesia.7,11,12 Pada penelitian ini yang akan 
dianalisis dari data mentah adalah Penyakit jantung 
koroner (PJK) yang penyakitnya berbeda dengan 
Stroke walaupun ada kemungkinan kesamaan 
faktor risiko.

Penyakit jantung koroner (PJK) 
didefinisikan jika pernah didiagnosis menderita 
PJK (angina pektoris dan/atau infark miokard) oleh 
dokter atau belum pernah didiagnosis menderita 
PJK tetapi pernah  mengalami gejala/riwayat nyeri

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
155
di dalam dada/rasa tertekan berat/ tidak nyaman 
di dada, nyeri/tidak nyaman di dada dirasakan di 
dada bagian tengah/dada kiri depan/menjalar ke 
lengan kiri, nyeri/tidak nyaman di dada dirasakan 
ketika mendaki/naik tangga/berjalan tergesa-gesa 
dan nyeri/tidak nyaman di dada hilang ketika 
menghentikan aktifitas/istirahat.

Diabetes mellitus (DM) didefinisikan jika 
pernah didiagnosis menderita kencing manis oleh 
dokter atau tidak pernah didiagnosis menderita 
kencing manis oleh dokter tetapi dalam satu bulan 
terakhir mengalami gejala sering lapar, sering haus 
dan sering buang air kecil dalam jumlah banyak, 
dan berat badan turun.

Stroke didefinisikan jika pernah didiagnosis 
menderita penyakit stroke oleh tenaga kesehatan 
(dokter/perawat bidan) atau belum pernah 
didiagnosis menderita penyakit stroke oleh nakes 
tetapi pernah mengalami secara mendadak keluhan 
kelumpuhan pada satu sisi tubuh atau kelumpuhan 
pada satu sisi tubuh yang disertai kesemutan atau 
baal satu sisi tubuh atau mulut menjadi mencong 
tanpa kelumpuhan otot mata atau bicara pelo  atau 
sulit bicara/komunikasi dan atau tidak mengerti 
pembicaraan.

Hipertensi ditentukan dari hasil pengukuran 
yang merujuk kriteria JNC VII 2003, yaitu hasil 
pengukuran tekanan darah sistolik ≥ 140 mmHg dan 
atau tekanan darah diastolic ≥ 90 mmHg atau dari 
hasil diagnosis dokter, gejala dan riwayat minum 
obat hipertensi.

Tekanan darah didapatkan dari 
rata-rata hasil pengukuran 2-3 kali menggunakan 
tensimeter digital.

Obesitas didefinisikan mengikuti 
kriteria Indeks Masa Tubuh (IMT) > 25 (WHO, 
Asia  Pasifik).

Obesitas didefinisikan mengikuti 
kriteria Indeks Masa Tubuh (IMT) > 25 (WHO, 
Asia  Pasifik).

Dikatakan sebagai obesitas sentral 
bila lingkar perut > 90 cm pada laki-laki dan> 80cm 
pada perempuan.

Status ekonomi dikategorikan memjadi 5 
kuintil berdasarkan indeks kepemilikan.

Kuintil 1,2 
dikategorikan status ekonomi rendah atau miskin 
dan kuintil 3,4,5 status ekonomi menengah keatas 
atau tidak miskin.

Akseptor KB hormonal adalah 
wanita usia subur (WUS) yang menggunakan alat 
Keluarga Berencana (KB) susuk, suntikan dan pil.

Kategori aktifitas fisik dalam Riskesdas 2013 
mengacu pada kriteria WHO Global Physical 
Activity Questionnaire (WHO GPAQ) dan terbagi 
menjadi 2 kategori yaitu aktifitas fisik aktif jika 
individu melakukan aktifitas fisik berat atau sedang 
atau kedua-duanya, sedangkan kriteria kurang aktif 
jika individu tidak melakukan aktifitas fisik berat 
atau sedang.

Gangguan mental emosional yaitu suatu 
istilah yang digunakan sejak Survei Kesehatan 
Rumah Tangga (SKRT) 1995 untuk menilai status 
mental penduduk.

Instrumen yang digunakan adalah 
Self Reporting Questionnaire (SRQ) yang terdiri 
dari 20 butir pertanyaan.

Dikatakan mengalami 
gangguan mental emosional jika responden 
menjawab minimal 6 atau lebih jawabanya.

Titik 
potong profil lipid sesuai dengan kriteria NCEP-
ATP III ditentukan dari hasil parameter total 
kolesterol  < 200 mg/dl, HDL ≥ 40 mg/dl, LDL < 
100 mg/dl, dan trigliserida < 150 mg/dl.7,11
HASIL
 Dari 722.329 data  responden berusia ≥ 
15 tahun di 33 provinsi di Indonesia didapatkan 
prevalensi penyakit jantung koroner secara 
nasional berdasarkan diagnosis dokter atau gejala 
sebesar 1,5%.

Tabel 1 menunjukkan karakteristik 
sampel dan prevalensi penyakit jantung koroner di 
Indonesia.

Tabel 1.

Tabel 1.

Karakteristik Sampel Berdasarkan Sosio Demografi, Perilaku, Biologik, dan Prevalensi Penyakit 
               Jantung Koroner di Indonesia Tahun 2013
Karakter Besar Sampel  Prevalensi PJK
(N, %) (N, %) 95% CI
Jenis Kelamin (n=722.329)
     Laki-laki 360.044 (49,8)  4.761 (1,3) (1,3 - 1,4)
     Perempuan 362.285 (50,2)  5.819 (1,6) (1,5 - 1,7)
Kelompok Usia (tahun) (n=722.329)
     15 - 24 167.006 (23,I) 1.098 (0,7) (0,6 - 0,7)
     25 – 34 175.365 (24,3) 1.636(0,9) (0,9 - 1,0)
     35 – 44 152.875 (21,2) 1.968(1,3) (1,2 -1,4)
     45 -  54 112.846 (15,6) 2.322(2,1) (1,9 - 2,2)

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
156
     55 - 64 65.136 (9,0) 1.850(2,8) (3,7 – 3,0)
     65 – 74 33.643 (4,6) 1.216(3,6) (3,3 – 3,9)
     ≥ 75 15.458 (2,1)        490(3,2) (2,8 – 3,6)
Status perkawinan (n=722.329)
    Belum menikah 174.309(24,1) 1.103(0,6) (0,6 – 07)
    Menikah, hidup bersama 495.517(68,6) 8.098(1,6) (1,6 – 1,7)
    Cerai hidup, pisah, cerai mati 52.503(7,3) 1.378(2,6) (2,4 – 2,8)
Pendidikan (n=722.329)
    Tidak sekolah 40.786(5,646) 1.148(2,8) (2,6 – 3,1)
    Tidak tamat SD 80.838(11,2) 1.826(2,3) (2,1 – 2,4)
    Tamat SD 214.803(29,7) 3.639(1,7)  (1,6 –1,8)
    Tamat SMP 153.107(21,2) 1.687(1,1) (1,0 – 1,2)
    Tamat SMA 185.798(25,7) 1.776(1,0) (0,9 – 1,0)
    Tamat PT/Akademik/Sederajat 46.996(6,5)        503(1,1) (0,9 – 1,2)
Pekerjaan (n=722.329)
    Sekolah, sedang mencari kerja 66.569(9,2) 451(0,7) (1,8 – 2,0)
    Bekerja 431.381(59,7) 5.828(1,3) (1,3 – 1,4)
    Tidak bekerja 224.377(31,1) 4299(1,9) (0,6– 0,8)
Tempat tinggal (n=722.329)
    Perkotaan 368.282(51,0) 5.083(1,4) (1,3 – 1,5)
    Perdesaan 353.847(49,0) 5.496(1,5) (1,4 - 1,5)
Status ekonomi (n=722.329)
    Terbawah 103.296 (14,3) 2.149(2,1) (1,9 – 2,2)
    Menengah bawah 134.888(18,7) 2.185(1,6) (1,5 – 1,7)
    Menengah 158.562(21,9) 2.211(1,4) (1.3 -  1,5)
    Menengah atas 175.470(24,3) 2.257(1,3) (1,2 – 1,4)
    Teratas 150.113(20,7) 1.778(1,2) (1,1 – 1,3)
Pola konsumsi sayur buah (n=722.329)
    Tidak konsumsi 150.485(20,8) 2.590(1,7) (2,6 – 4,2)
    1 – 2 porsi/hari 456.189(63,2) 6.171 (1,4) (1,3 -1,4)
    3 – 4 porsi/hari 99.061(13,7) 1.551(1,5) (1,4 – 1,7)
    ≥ 5 porsi 16594(2,3)        267(1,6) (1,3 – 1,9)
Aktifitas fisik (n=722.329)
     Kurang 163.175 (22,6) 3.167(1,9) (1,8 – 2,1)
     Cukup 559.154(77,4) 7.413 (1,3) (1,3 – 1,4)
Konsumsi minuman berkafein selain
Kopi (n=722.329)
     ≥ 1X/hari 42.798(5,9) 759(1,8) (1,6 – 2,0)
     1-6X/minggu 92,625(12,8) 1.285(1,4) (1,2 – 1,5)
     < 3X/bulan 586.906(81,3) 8.536(1,5) (1,4 – 1,5)
Konsumsi minum kopi (n=722.329)
      ≥ 1X/hari 235.400(32,6) 3.240(1,4) (1,4 – 1,5)
      1-6X/minggu 131.497(18,2) 1.638(1,2) (1,1 – 1,4)
      <3X/bulan 355.432(49,2) 5.701(1,6) (1,5 – 1,7)
Merokok (n=722.329)
      Tiap hari 201.084(27,8) 2.442(1,2) (1,1 – 1,3)
      Kadang-kadang 40.456(5,6) 537(1,3) (1,2 – 1,5)
      Mantan perokok 32.795(4,6) 1298(4,0) (3,6 – 4,3)
      Tidak pernah 447.993(62,0) 6.302(1,4) (1,4 – 1,5)

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
157
Status gizi berdasarkan IMT (n=712.789)
      Obesitas 174.879(24,6) 3.019(1,7) (1,6 – 1,8)
      Berat badan lebih 117.961(16,5) 1.512(1,3) (1,2 – 1,4)
      Kurus 92.963(13,0) 1.553(1,7) (1,5 – 1,8)
      Normal 326.986((45,9) 4.189(1,3) (1,2 – 1,4)
Status gizi berdasarkan LP(n=688.504)
      Obesitas sentral 183.158(26.6) 3.467(1,9) (1,8 – 2,0)
      Tidak obesitas 505.346(73,4) 6.616(1,3) (1,3 – 1,4)
Gangguan mental emosional (n=722.329)
      Ya 46.008(6,4) 3.786(8,2) (7,8 – 8,7)
      Tidak 676.322(93,6) 6.794(1,0) (1,0 – 1,0)
Tekanan darah (n=722.329)
      Hipertensi 62.943(8,7) 3.464(5,5) (5,2 – 5,8)
      Normal 659.386(91,3) 7.155(1,1) (1,0 – 1,1)
Diabetes Melitus(DM) (n=722.329)
      Penyandang DM 15.221(2,1) 1.407(9,2) (8,5 – 0,1)
      Bukan penyandang DM 707.107(97,9) 9.172(1,3) (1,2 – 1,5)
Stroke (n=722.329)
      Ya 8.706(1,2) 760(8,7) (7,9 – 9,7)
      Tidak 713.624(98,8) 9.820(1,4 ) (1,3 – 1,4)
Hasil Profil lipid (n=58.044)
      Abnormal 49.683(85,6) 885(1,8) (1,6 – 2,0)
      Dalam batas normal 8.362(14,4) 112(1,3) (1,0 - 1,8)
Riwayat KB hormonal (n=126.387)
      Akseptor KB hormonal 108.520(85,9) 1.245(1,1) (1,0 – 1,3)
      Bukan Akseptor KB hormonal 17.867(14,1) 278(1,6) (1,3 – 1,9)
Keterangan: n= besar sampel dengan weight
           Tabel 1 menunjukkan secara karakteristik 
besar sampel perempuan sedikit lebih banyak dari 
laki-laki.

Prevalensi penyakit jantung koroner 
terlihat lebih besar pada perempuan yaitu 1,6% 
sedangkan pada laki-laki hanya 1,3%.

Sebagian 
besar responden sudah menikah/hidup bersama 
dan bekerja, namun pada kelompok penderita 
jantung koroner prevalensi tidak bekerja 
lebih tinggi dibanding bekerja.

Kemungkinan 
responden tidak bekerja karena penyakit jantung 
koroner.

Prevalensi responden cerai hidup/mati 
atau pisah pada kelompok penderita jantung 
koroner lebih tinggi dibanding hidup bersama 
atau belum menikah, kemungkinan perceraian 
meningkat akibat penyakit jantung koroner atau 
karena usia responden sudah tua dan pasangan 
hidup sudah terlebih dahulu meninggal.

Responden tinggal di perdesaan lebih sedikit dari 
perkotaan namun penderita jantung koroner lebih 
banyak di perdesaan.

Prevalensi penyakit jantung 
koroner terbanyak pada usia 25-34 tahun, disusul 
kelompok usia 15-24 tahun dan terlihat  penyakit 
jantung koroner sudah muncul pada kelompok 
usia muda sebesar 0,7% dan prevalensi meningkat 
pada usia 45-54 tahun ke atas tertinggi pada usia 
65-74 tahun sebesar 3,6% di atas usia 75 tahun 
menurun sedikit jadi 3,2%.

Prevalensi penderita 
jantung koroner lebih tinggi pada kelompok 
tidak sekolah dan tidak tamat SD dibandingkan 
kelompok tamat SD ke atas.

Jumlah responden 
pada indeks kepemilikan menengah keatas 
lebih banyak dibanding yang lain, namun pada 
kelompok penderita jantung koroner prevalensi 
penderita jantung koroner paling tinggi pada 
kuintil indeks kepemilikan terbawah sebesar 2,1% 
disusul menengah bawah yaitu sebesar 1,6%; 
kemungkinan akibat penyakit jantung koroner 
maka status ekonomi menjadi menurun.

Pada umumnya responden makan sayur 
buah 1-2 porsi saja, namun pada kelompok 
penderita jantung koroner prevalensi konsumsi 
sayur buah > 5 porsi lebih tinggi, kemungkinan 
perilaku meningkatkan konsumsi sayur buah 
akibat penyakit jantung koroner.

Jumlah

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
158
responden yang beraktifitas cukup lebih tinggi 
dibanding aktifitas kurang, namun prevalensi 
penderita jantung koroner lebih tinggi pada yang 
beraktifitas kurang, kemungkinan akibat penyakit 
jantung koroner tidak dapat melakukan aktifitas 
fisik.

Sebagian besar  responden tidak setiap 
minggu mengonsumsi minuman berkafein selain 
kopi.

Prevalensi penyakit jantung koroner lebih 
tinggi dari pada yang mengkonsumsi minuman 
yang berkafein selain kopi setiap hari.

Prevalensi 
penyakit jantung koroner pada responden yang 
mengkonsumsi kopi minimal tiap minggu hampir 
sama dengan yang mengkonsumsi kopi minimal 
tiap bulan.

Jumlah responden yang tidak pernah 
merokok lebih tinggi dibanding yang merokok/
pernah,  namun yang merokok tiap hari cukup 
tinggi sebesar 27,8%.

Terlihat prevalensi penderita 
jantung koroner lebih tinggi pada mantan perokok 
(4%).

Terlihat prevalensi penderita 
jantung koroner lebih tinggi pada mantan perokok 
(4%).

Hal ini kemungkinan akibat penderita 
jantung koroner menghentikan perilaku merokok.

Responden dengan Indeks Massa Tubuh 
normal lebih banyak, demikian juga dengan 
lingkar perut normal (tidak ada obesitas sentral).

Status gizi berdasarkan indeks massa tubuh 
(IMT) menunjukkan prevalensi penderita jantung 
koroner lebih tinggi pada yang obesitas dan 
kurus.

Demikian juga jika dilihat berdasarkan 
lingkar perut (LP) prevalensi PJK lebih tinggi 
pada populasi dengan obesitas sentral.

Prevalensi 
PJK lebih tinggi pada kelompok yang mengalami 
gangguan mental emosional, ada kemungkinan 
penyakit jantung koroner meningkatkan gangguan 
mental emosional atau sebaliknya.

Terlihat prevalensi penderita jantung 
koroner lebih tinggi pada yang hipertensi 
dibanding yang tidah hipertensi, lebih tinggi pada 
diabetes mellitus dibanding yang tidak diabetes 
mellitus.

Demikian juga prevalensi penderita 
jantung koroner lebih tinggi pada penderita 
stroke dibanding tidak stroke, pada hasil profil 
lipid abnormal dibandingkan dengan dalam batas 
normal.

Jumlah responden yang merupakan 
akseptor KB hormonal lebih banyak tetapi yang 
menderita penyakit jantung koroner lebih banyak 
pada bukan akseptor  KB hormonal.

Selanjutnya dilakukan analisis bivariat 
dan berdasarkan Tabel 1 variabel yang dianggap 
kejadiannya dipicu akibat menderita penyakit 
jantung koroner tidak dimasukkan.

Hal ini 
merupakan salah satu keterbatasan metode potong 
lintang yang mengumpulkan data pada satu titik 
waktu.

Tabel 2.

Tabel 2.

Hubungan Faktor Risiko dengan Kejadian Penyakit Jantung Koroner
Karakteristik OR 95 % CI Signifikan (p)
Jenis kelamin
Perempuan 1,22 1,17- 1,27 0,0001
Laki-laki 1
Usia
≥ 40 tahun 2,72 2,61-2,83 0,0001
< 40 tahun 1
Tingkat sosial ekonomi
Miskin 1,42 1,36-1,48 0,0001
Tidak miskin 1
Pendidikan
Tidak sekolah – tamat SD 3,48 2,46-4,93 0,0001
Tamat SMP-Tamat SMA 1,94 1,37-2,76 0,0001
Tamat PT/Akademi/sederajat 1
Diabetes melitus
     Ya 7,75 7,31-8,22 0,0001
     Tidak 1
Hipertensi
     Ya 5,34 5,12-5,67 0,0001
     Tidak 1
Stroke
     Ya 6,86 6,35-7,40 0,0001

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
159
     Tidak 1
Status gizi menurut IMT
Berat badan lebih-obesitas 1,21 1,16-1,26 0,0001
Kurus 1,31 1,24-1,39 0,0001
Normal 1
Status gizi berdasarkan LP
Obesitas sentral 1,45 1,40-1,52 0,0001
Tidak obesitas 1
Gangguan mental emosional
Ya 8,84 8,48-9,21 0,0001
Tidak 1
Kebiasaan merokok
     Perokok/mantan perokok 1,11 1,07-1,15 0,0001
     Bukan perokok 1
Konsumsi kopi
≥ 1 kali/hari 0,86 0,82-0,89 0,0001
1-6 kali /minggu 0,77 0,73-0,82 0,0001
 ≤ 3 kali/bulan 1
Konsumsi kafein selain kopi
     Minimal tiap minggu 1,04 0,99-1,09 0,129
     Minimal tiap bulan 1
Konsumsi Buah-sayur
      Tidak pernah konsumsi/hari 1,07 0,94-1,22 0,298
      1-2 porsi/hari 0,84 0,74-0,95 0,006
3-4 porsi/hari 0,97 0,85-1,11 0,688
≥ 5 porsi/hari 1
Profil lipid
Abnormal 1,34 1,10-1,63 0,004
Normal 1
Aktifitas Fisik
Kurang aktif 1,47 1,41-1,54 0,0001
Aktif 1
Akseptor KB hormonal
Ya
Tidak
0,73
1
0,64-0,84 0,0001
 Pada Tabel 2 menunjukkan bahwa jenis 
kelamin perempuan, usia ≥ 40 tahun, status 
ekonomi rendah, tingkat pendidikan rendah, 
diabetes mellitus, hipertensi, stroke, obesitas, 
kurus, gangguan mental emosional, kebiasaan 
merokok, profil lipid abnormal dan aktifitas fisik 
yang kurang aktif  mempunyai hubungan bermakna 
dalam meningkatkan risiko  kejadian penyakit 
jantung koroner.

Jenis kelamin perempuan 
berisiko 1,22 kali  terjadinya penyakit jantung 
koroner  dengan p=0,0001, 95% CI 1,17-1,27.

Umur pada analisis bivariat ini dikelompokkan 
menjadi dua kelompok yang berusia kurang 
dari 40 tahun dan yang berusia sama atau lebih 
dari 40 tahun sesuai WHO tahun 2000 yang 
menyatakan bahwa usia ≥ 40 tahun berisiko tinggi 
mengalami penyakit tidak menular.

Responden 
berusia sama atau lebih dari 40 tahun berisiko 
2,72 kali dibandingkan yang berusia kurang dari 
40 tahun dengan p= 0,0001, 95% CI 2,61-2,83.

Status ekonomi rendah (miskin) pada kuintil 1 
dan 2, lebih berisiko dibandingkan dengan status 
ekonomi menengah ke atas (tidak miskin) pada 
kuintil 3,4 dan 5.

Responden dengan pendidikan 
kurang tinggi lebih berisiko dibandingkan yang 
berpendidikan tinggi.

Diabetes mellitus berisiko 
7,75 kali dibanding yang tidak.

Hipertensi 
berisiko 5,34 kali dibanding yang tidak.

Riwayat 
stroke berisiko 6,86 kali dibanding yang tidak.

Riwayat 
stroke berisiko 6,86 kali dibanding yang tidak.

Responden dengan status gizi menurut Indeks

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
160
Massa Tubuh (IMT) dalam kategori berat badan 
lebih-obesitas berisiko 1,21 dibanding  normal, 
yang kurus berisiko 1,31 dibanding  normal.

Responden dengan status gizi berdasarkan 
lingkaran perut (LP) dalam kategori obesitas 
sentral berisiko 1,45 kali dibanding yang tidak 
obesitas.

Gangguan mental emosional berisiko 
8,84 kali dibanding yang tidak.

Perokok/mantan 
perokok berisiko 1,11 dibanding bukan perokok.

Konsumsi kafein selain kopi dengan frekuensi 
minimal tiap minggu berisiko 1,04 dibanding 
yang minimal tiap bulan.

Profil lipid abnormal 
meningkatkan risiko 1,34 kali dibanding yang 
normal.

Demikian juga beraktifitas fisik kurang 
meningkatkan risiko 1,47 kali dibanding  yang 
baik.

Selanjutnya dilakukan uji kompleks sampel 
logistik regressi multivariat.

Selanjutnya dilakukan uji kompleks sampel 
logistik regressi multivariat.

Berdasarkan tabel 2 
maka variabel dengan nilai p < 0,25 dimasukkan 
dalam analisis multivariat kecuali profil lipid 
dan akseptor KB hormonal karena pengambilan 
sampel darah pada saat Riskesdas 2013 dan 
WUS hanya subsampel.

Setelah dilakukan uji 
confounding dan interaksi didapatkan hasil akhir 
regresi logistik ganda pada Tabel 3.

Tabel 3.

Tabel 3.

Faktor Risiko Penyakit Jantung Koroner pada Usia ≥ 15 Tahun
Variabel ORadjusted 95  % C.I pMin Max
Tingkat sosial ekonomi
Tidak miskin 1
Miskin 1,30 1,24 1,36 0,0001
Obesitas sentral
Tidak 1
Ya 1,45 1,33 1,58 0,0001
Pendidikan
Tamat SMP ke atas 1
Tidak sekolah-tamat SD 1,48 1,38 1,59 0,0001
Jenis kelamin
Laki-laki 1
Perempuan 1,61 1,43 1,81 0,0001
Kebiasaan merokok
Tidak 1
Ya 1,61 1,43 1,80 0,0001
Usia 
< 40 tahun 1
≥ 40 tahun 3,70 3,04 4,50 0,0001
Stroke
Tidak 1
Ya 3,73 3,14 4,44 0,0001
DM
Tidak 1
Ya 8,43 7,14 9,95 0,0001
Gangguan mental emosional
Tidak 1
Ya 9,58 8,89 10,33 0,0001
Hipertensi
Tidak 1
 Ya 10,09 8,48 12,01 0,0001

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
161
PEMBAHASAN
 Prevalensi penyakit jantung koroner pada 
status ekonomi rendah atau miskin sebedar 3,7 %.

Responden dengan status sosial ekonomi miskin 
berisiko mengalami penyakit jantung koroner 
1,3 kali.

Ini sesuai dengan hasil penelitian Wong4 
yang menunjukkan 80% kematian akibat penyakit 
jantung terjadi di negara-negara berpenghasilan 
rendah dan menengah.

Risiko responden dengan 
tingkat pendidikan rendah juga lebih tinggi 
dibandingkan pendidikan tinggi (OR 1,48 95%CI 
1,38-1,59).

Berbagai penelitian berhasil membuktikan 
faktor-faktor risiko antara lain obesitas, merokok, 
profil lipid abnormal, dan lain-lain.

The Global 
Burden of Metabolic Risk Factors for Chronic 
Diseases Collaboration yang menggabungkan 97 
penelitian kohor dengan lokasi penelitian tersebar 
di seluruh dunia, juga membuktikan bahwa 
obesitas mempunyai hubungan yang signifikan 
terhadap penyakit jantung koroner antara obesitas 
dengan penyakit jantung koroner.

Penelitian ini 
menghasilkan pooled RR sebesar 1,69 (95% CI 
1,58-1,81).10
 Adanya hasil yang signifikan antara 
obesitas dengan penyakit jantung koroner adalah 
karena obesitas dapat meningkatkan tekanan 
darah, kadar trigliserida, kolesterol, resistensi 
glukosa, serta penggumpalan darah.

Peningkatan 
tekanan darah membuat pembuluh darah rentan 
untuk mengalami penebalan dan penyempitan.

Hal tersebut jika terjadi pada arteri koroner akan 
menimbulkan penyakit jantung koroner.

Begitu 
pula, jika terjadi peningkatan kadar trigliserida 
dan kolesterol, hal ini akan memicu munculnya 
thrombosis plak pada pembuluh darah.

Hal ini juga 
dapat menimbulkan penyakit jantung koroner.

Dalam penelitian ini prevalensi penyakit 
jantung koroner di Indonesia tahun 2013 menurut 
jenis kelamin lebih tinggi pada perempuan 
dibanding laki-laki  yaitu 1,6% dibanding 1,3% 
(Tabel 1).

Perempuan setelah memperhitungkan 
faktor-faktor lain berisiko 1,61 kali (95% 
CI 1,43-1,81, p 0,0001) dibanding laki-laki.

Dikatakan sebelum menopause kemungkinan 
perempuan terkena penyakit jantung koroner 
lebih kecil dibanding laki-laki.

Namun dengan 
meningkatnya usia dan menurunnya estrogen 
setelah menopause, risiko penyakit jantung 
meningkat.

Dulu akibat pengaruh budaya dan cara 
pandang kesadaran akan pentingnya kesehatan 
jantung pada kaum perempuan masih rendah, 
masyarakat sering beranggapan bahwa kesehatan 
suami atau kaum laki-laki lebih penting.

Masih 
banyak pula profesional medis yang beranggapan 
bahwa penyakit kardiovaskuler adalah penyakit 
kaum laki-laki, sehingga penyakit kardiovaskuler 
pada perempuan yang gejala-gejalanya tidak 
khas, seringkali terlewatkan.

Fakta sekarang 
American Heart Association (AHA) mengatakan 
lebih dari 1/3 perempuan dewasa menderita salah 
satu bentuk penyakit kardiovaskuler terutama 
penyakit jantung koroner dan jumlah kematian 
pada perempuan melebihi laki-laki.

Tahun 2012, 
sekitar 56% penyebab kematian perempuan adalah 
penyakit kardiovaskuler dan  terbanyak adalah 
penyakit jantung koroner.13,14,15
 Prevalensi perokok pada penyakit jantung 
koroner  kalau tiap hari  merokok 1,2%,  kadang-
kadang merokok 1,3% dan mantan perokok  4%.

Merokok juga dibagi 2 kelompok yaitu perokok/
mantan prokok dan bukan perokok.

Merokok 
setelah dikontrol dengan faktor-faktor lain berisiko 
1,61 kali (95% CI 1,43-1,80).

Merokok 
setelah dikontrol dengan faktor-faktor lain berisiko 
1,61 kali (95% CI 1,43-1,80).

Ini sesuai dengan 
penelitian yang dilakukan oleh Rachel R Huxlley 
& Mark Woodward melakukan penelitian meta-
analisis meliputi penelitian kohor yang dilakukan 
di seluruh dunia.

Penelitian tersebut membuktikan 
bahwa orang yang merokok memiliki risiko lebih 
besar untuk menderita penyakit jantung koroner 
dengan pooled RR sebesar 1,25 (95%CI 1.12-
1.39, p<0,0001).16 
 Menurut  World   Heart  Federation    tembakau 
yang dikandung dalam rokok dapat menyebabkan 
penurunan kadar oksigen yang dialirkan oleh

1.39, p<0,0001).16 
 Menurut  World   Heart  Federation    tembakau 
yang dikandung dalam rokok dapat menyebabkan 
penurunan kadar oksigen yang dialirkan oleh 
darah dan menyebabkan darah cenderung mudah 
menggumpal.

Gumpalan darah yang terbentuk di 
arteri ini dapat menyebabkan penyakit jantung 
koroner dan juga  stroke serta kematian mendadak.

Literatur lain dari Heart Foundation menyatakan 
bahwa tembakau memiliki efek patofisiologi 
terhadap jantung, sistem pembekuan darah, dan 
metabolisme lipoprotein.

Merokok meningkatkan 
pembentukan plak koroner dan mendorong 
terjadinya thrombosis koroner.

Merokok juga 
dapat meningkatkan kebutuhan oksigen oleh otot 
jantung dan menurunkan kemampuan darah untuk 
mengangkut oksigen.9  
 Berdasarkan kelompok usia, didapatkan

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
162
responden yang berusia ≥  40 tahun  berisiko 
2,72 kali dibanding < 40 tahun (tabel 2).

Setelah 
dikontrol dengan variable lain risiko usia ≥  40 
tahun meningkat menjadi 3,70 kali (95% CI 3,04 
– 4,50, p 0,0001).

Usia merupakan faktor risiko 
penyakit jantung koroner dimana penambahan 
usia akan meningkatkan risiko terjadinya  
penyakit jantung koroner.

Semakin tua usia maka 
semakin besar timbulnya plak yang menempel 
di dinding dan menyebabkan gangguan aliran 
darah yang melewatinya.

Penelitian Donald di 
RSU Pirngadi di Medan menunjukkan hasil 
yang sama yaitu penyakit jantung koroner 
terdapat lebih banyak pada kelompok usia ≥ 40 
tahun.17 Faktor usia terbukti berhubungan dengan 
kematian akibat penyakit jantung koroner.

Tanda 
dan gejala penyakit jantung koroner banyak 
dijumpai pada individu-individu dengan usia 
yang lebih tua.

Faktor usia juga berhubungan 
dengan kadar kolesterol yaitu kadar kolesterol 
total akan meningkat dengan bertambahnya 
umur.

Kandungan lemak berlebihan dalam darah 
pada hiperkolesterolemia dapat menyebabkan 
penimbunan kolesterol pada dinding pembuluh 
darah sehingga pembuluh darah akan menyempit 
dan akibatnya tekanan darah akan meningkat dan 
terjadilah penyakit jantung koroner.18  Stroke juga 
terlihat  merupakan salah satu faktor risiko yang 
dominan.

Prevalensi penyakit jantung koroner 
pada stroke 8,7% (tabel 1) dan setelah dikontrol 
dengan faktor-faktor lain, stroke meningkatkan 
risiko penyakit jantung koroner 3,73 kali (95% CI 
3,14 - 4,44) dibandingkan tanpa stroke.

Dikatakan 
Stroke selain merupakan salah satu dari penyakit 
kardiovaskuler juga merupakan faktor risiko dari 
penyakit kardiovaskuler lainnya.

9 Sebaliknya 
penyakit jantung koroner juga merupakan faktor 
risiko dari stroke dengan OR 3,13 (95% 2,72 - 
3,60).12
 Terlihat juga diabetes melitus merupakan 
salah satu faktor risiko yang dominan.

Prevalensi 
penyakit jantung koroner pada diabetes melitus 
9,2 % (Tabel 1) dan setelah  dikontrol dengan 
faktor-faktor lain, berisiko 8,43 kali (95% CI 
7,14-9,95) dibandingkan tidak diabetes melitus.

Hasil analisis data Riskesdas 2007 menunjukkan 
diabetes melitus meningkatkan risiko penyakit 
jantung, termasuk penyakit jantung koroner, 
sebesar 4,06 kali (95% CI 3,79-4,35) dibandingkan 
yang tidak diabetes melitus.19 Menurut American 
Heart Association pada Mei 2012, paling kurang 
65% penderita diabetes melitus mengakibatkan 
penyakit jantung atau stroke.

Selain itu, orang 
dewasa yang menderita diabetes melitus berisiko 2 
sampai 4 kali lebih besar  terkena penyakit jantung  
dari pada orang yang tidak menderita diabetes 
mellitus.20,21 Orang dengan diabetes cenderung 
lebih cepat mengalami degenerasi jaringan dan 
disfungsi dari endotel sehingga timbul proses 
penebalan membrane basalis dari kapiler dan 
pembuluh darah arteri koronaria sehingga  terjadi 
penyempitan aliran darah  ke jantung.22 
 Dengan adanya resistensi glukosa, 
maka glukosa dalam darah akan meningkat dan 
hal ini akan meningkatkan kekentalan darah.

Kecenderungan untuk terjadinya aterosklerosis 
pun meningkat dan dapat mengakibatkan 
terjadinya penyakit jantung koroner.

Prevalensi gangguan mental emosional 
pada penyakit jantung koroner 8,2%, setelah 
dikontrol dengan faktor-faktor lain, berisiko 
dengan OR 9,58 kali (95% CI 8,89-10,33).

Gangguan mental emosional di sini termasuk 
depresi dan stress.

Gangguan mental emosional di sini termasuk 
depresi dan stress.

WHO memprediksi pada tahun 
2030, depresi akan mengakibatkan disabilitas 
pada penyakit kronis termasuk penyakit jantung 
kronis.

Depresi didentifikasi sebagai masalah 
utama yang berdiri sendiri dan merupakan faktor 
risiko  mayor penyebab disabilitas pada penyakit 
kronis.

Depresi meningkatkan angka morbiditas 
dan mortalitas penyakit seperti kanker, diabetes, 
penyakit jantung, dan stroke.

Juga termasuk faktor 
gaya hidup yang buruk seperti merokok, kurang 
aktivitas fisik, diet yang salah dan mekanisme

dan mortalitas penyakit seperti kanker, diabetes, 
penyakit jantung, dan stroke.

Juga termasuk faktor 
gaya hidup yang buruk seperti merokok, kurang 
aktivitas fisik, diet yang salah dan mekanisme 
biologis.23  Penelitian lain menunjukkan bahwa 
pada sebagian orang, yakni mereka yang depresi 
dan terisolasi secara sosial, atau mereka yang 
tidak memiliki dukungan sosial yang berkualitas 
berisiko tinggi terkena penyakit jantung koroner.24 
 Prevalensi penyakit jantung koroner 
pada yang hipertensi 5,5%.

Responden dengan 
hipertensi berisiko mengalami penyakit jantung 
koroner 10,09 kali (95% CI 8,48-12,01) 
dibandingkan dengan yang tidak hipertensi.

Penelitian Townsend tahun 2012 membuktikan 
50% penyakit jantung koroner di negara 
berkembang terjadi disebabkan oleh hipertensi.

Ini 
menunjukkan hipertensi adalah salah satu faktor 
risiko utama terjadinya penyaki jantung koroner.25 
Sementara kejadian hipertensi di Indonesia

Faktor Risiko Dominan Penyakit Jantung Koroner di Indonesia ... (Lannywati Ghani, Made Dewi Susilawati dan Harli Novriani )
163
menurut WHO sebesar 42,7% pada laki-laki dan 
39,2 % pada wanita.26 Hipertensi tidak terkontrol 
cenderung mengakibatkan komplikasi lain 
seperti stroke, ateroslerosis, aneurisma, sindroma 
metabolik, penyakit ginjal.

Penyakit jantung 
koroner juga mempunyai komplikasi-komplikasi 
tersendiri seperti aritmia, gagal jantung kongestif, 
infark miokardial, dan kematian.

18 Ini berarti 
pasien penyakit jantung koroner yang disertai 
hipertensi harus diberi perhatian yang lebih baik 
karena pasien ini mendapat komplikasi dari 
kedua penyakit tersebut.

Oleh karena itu sangat 
dianjurkan membuat penelitian mengenai berapa 
besar jumlah pasien penyakit jantung koroner 
yang disertai dengan hipertensi dan meneliti 
karakteristik hipertensi jadi pasien penyakit 
jantung koroner.

Hasil analisis ini menunjukkan faktor 
risiko dominan yang berpengaruh terhadap 
penyakit jantung koroner adalah dari yang 
terbesar sampai terkecil  kekuatan hubungannya 
adalah hipertensi, gangguan mental emosional, 
diabetes mellitus, stroke, umur ≥ 40 tahun, riwayat 
kebiasaan merokok, jenis kelamin perempuan, 
tingkat pendidikan tidak sekolah-tamat SD, 
obesitas sentral, dan tingkat sosial ekonomi miskin 
atau rendah.

KESIMPULAN
 Faktor risiko penyakit jantung koroner 
adalah hipertensi, gangguan mental emosional, 
diabetes melitus, stroke, usia ≥ 40 tahun, kebiasaan 
merokok, perempuan, tingkat pendidikan rendah, 
obesitas sentral, dan tingkat sosial ekonomi rendah 
dengan ORadjusted berkisar dari 1,30 hingga 10,09.

Faktor risiko dominan penyakit jantung koroner 
di Indonesia adalah hipertensi, gangguan mental 
emosional, dan diabetes melitus.

KETERBATASAN
 Hasil analisis dan kesimpulan disampaikan 
dengan keterbatasan sebagai berikut: desain 
penelitian adalah potong lintang sehingga 
kurang kuat untuk menyatakan suatu hubungan 
sebab akibat.

Penyakit jantung koroner, diabetes 
mellitus, dan stroke didefinisikan menurut hasil 
wawancara bukan menurut hasil pemeriksaan 
fisik dan penunjang.

UCAPAN  TERIMA  KASIH
 Kami mengucapkan terima kasih kepada 
Kepala Balitbang Kesehatan RI, Kepala Pusat 
Penelitian dan Pengembangan Sumber Daya dan 
Pelayanan Kesehatan,  tim riset kesehatan dasar 
(RISKESDAS) 2013  sehingga kami memperoleh 
data-data yang bisa dianalisis lebih lanjut.

Ucapan 
terima kasih kami juga kepada semua pihak yang 
telah membantu dan membimbing sehingga artikel 
dapat diselesaikan.

DAFTAR RUJUKAN
1.

National Institute of Health( NIH).

About health 
topics [cited 2016 May 26].

Available from: 
http://www.nhbli.nih.gov//health-topics/topics/
cad
2. WHO 2013 About  Cardiovascular diseases.

World Health OrganizLation.

Geneve.

[cited 
2016 Apr 26].

Available from: http://www.who.

int/cardiovascular_diseases/about_cvd/en/
3.

Cardiovascular Disease [cited 2015 Sep 18].

Available from: http://www.who.int/cardiovas-
cular_ diseases/en/.

4.

Wong,N.D.(2014).

Epidemiological studies of 
CHD and the evolution of preventive cardiology, 
Heart Diseases Prevention Program, Division 
of Cardiology, University of California, 
Irvine, C240 Medical Sciences.

University of 
California, Irvine,CA 92697, USA.ndwong@
uci.edu V olume 11:276-289
5.

Citrakesumasari.

Model Prediksi Suspek 
Penyakit Jantung Koroner pada individu dan 
Masyarakat di Indonesia [disertasi].

Makassar: 
Hasanuddin Univ.;2009.

6.

MOH,National Institut of Health Research 
and Development, Center for Community 
Empowerment, Health Policy and Humanites 
2015.

Indonesia: Sample Registration System 
2014.

NIHRD Library Cataloguing in 
Publication Data.

7.

Badan Penelitian dan Pengembangan Kesehatan.

NIHRD Library Cataloguing in 
Publication Data.

7.

Badan Penelitian dan Pengembangan Kesehatan.

Pokok-Pokok Hasil Riskesdas Indonesia tahun 
2013.

Jakarta: Lembaga Penerbit Balitbangkes; 
2014.

8.

Global Atlas on Cardiovascular Disease 
Prevention and Control.

Mendis S, Puska P,  
Norrving B editors  World Health Organization

2013.

Jakarta: Lembaga Penerbit Balitbangkes; 
2014.

8.

Global Atlas on Cardiovascular Disease 
Prevention and Control.

Mendis S, Puska P,  
Norrving B editors  World Health Organization  
in Collaboration with the World Heart Federation 
and World Stroke Organization, Geneve; 2011.

Buletin Penelitian Kesehatan, V ol.

44, No.

3, September  2016 : 153 - 164
164
9.

Cardiovascular Disease Risk Factor: [cited  
2015 Sep 18].

Available from: http://www.

world-heart-federation.org/fileadmin/
user_upload/documents/Fact_sheets/2012/
PressBackgrounderApril2012RiskFactors pdf.

10.

Lu Y , Hajifathalian K, Ezzati M, Woodward 
M, Rimm EB, Danaei G. Metabolic mediators 
of the effects of body-mass index, overweight, 
and obesity on coronary heart disease and 
stroke: a pooled analysis of 97 prospective 
cohorts with 1,8 million participants.

Lancet.

2014;383(9921):970-83.

11.

Badan Penelitian dan Pengembangan Kesehatan 
Departemen Kesehatan RI.

Riset Kesehatan 
Dasar 2013.

Pedoman Pengisian Kuesioner.

Jakarta: Badan Penelitian dan Pengembangan 
Kesehatan Kementerian Kesehatan RI; 2013.

12.

Ghani L, Mihardja LK, Delima.

Faktor risiko 
dominan penderita stroke di Indonesia.

Buletin 
Penelitian Kesehatan.

2016; 44(1): 49-58.

13.

Lloyd Jones DM.

Buletin 
Penelitian Kesehatan.

2016; 44(1): 49-58.

13.

Lloyd Jones DM.

Cardiovasculer risk prediction: 
basic concepts, current status and future 
directions.

Circulation.

2010;121:1768-1777.

14.

Greenlan P, Alpert JS, Biller GA, Benyamin 
EJ, Budoff MJ, Fayad ZA,etal.

ACCF/AHA.

Guidelines for assessment For Cardiovaskuler 
risk in asymptomatic adults: A report of the 
American College of Cardiology Foundation/ 
American Heart Association Task Force on 
practice guidelines.

J Am Coll Cardio  2010; 
56(25): e50-103.

15.

Salam T, Watson KE.

Predictor of cardiovascular 
risk in women.

Womens Health 2013; 9(5): 491-
498.

16.

Huxley RR, Woodward M. Cigarette smoking as 
a risk factor for coronary heart disease in women 
compared with men: a systematic review and 
meta-analysis of prospective cohort studies.

The 
Lancet.

2011; 378(9799):1297-1305.

17.

The 
Lancet.

2011; 378(9799):1297-1305.

17.

Nababan D. Hubungan faktor risiko dan 
karakteristik penderita dengan kejadian 
penyakit jantung koroner  di RSU dr Pirngadi 
Medan tahun; 2008 (tesis).

Medan.

Universitas 
Sumatera Utara, 2008.

18.

Kumar P., 2012.

Coronary Artery Disease 
Clinical Medicine Eight Edition, International 
Edition, Spain.

723-724.

19.

Delima, Mihardja L, Siswoyo H. Prevalensi dan 
faktor determinan penyakit jantung di Indonesia.

Buletin Penelitian Kesehatan.

2009; 37(3): 142-
159.

20.

National Diabetes Education Program (NDEP).

The Link Between Diabetes and Cardiovascular 
disease  [Cited 2012 Oct 31 Oct) .

Available from 
http://indep.nih.gov/media/CVD FactSheet pdf.

21.

Shahab A. Komplikasi klinik DM penyakit 
jantung koroner.

Dalam Sudyo AW, Setiyohadi 
B, Alwi I, Simadibrata KM, Setiati S,editor 
(penyunting).

Buku Ajar Ilmu Penyakit Dalam 
Jilid III Edisi ke -4.

Jakarta Departemen Ilmu 
Penyakit Dalam FKUI; 2007.

22.

Jakarta Departemen Ilmu 
Penyakit Dalam FKUI; 2007.

22.

Lewis, at al, Medical Surgical Nursing: 
Assesment and Management of Clinical 
Problems.

Seven Edution.

volume 2.

Mosby 
Elsevier;2011 
23.

Boris  V , William DR. and Rahn K. 2013.

Depression and Chronic Diseases.

It is time 
for a synergistic.

Mental Health and Primary 
care approud Prime Care  Companion CNS 
Discord.15 (2):PCC 12r01468.

24.

Lippi G.Montagnana M, Favaloro EJ, Franchini 
M. Mental depression and Cardiovascular 
disease.Amultceted, bidirectional association.

Semin Thronb Hemost 2009; 35: 325-336.

25.

Townsend,N.

Medical Risk Faktor Coronary 
Heart Statistics Acomoendium of Health 
Statistic.

Departement of Health.

Public, 
University Oxford;2013.

26.

World Health Organization, WHO, 2013.Car -
diovascular Diseases.

[Cited 2016 Mar 16].

Available from:http://www.who.int/mediacen -
tre/factsheds/fs317/en

In [ ]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

# Query

In [ ]:
question = input('Question: ')
response = rag_chain.invoke(question)
md(response)

Question: apa gejala jantung koroner


Berdasarkan konteks yang diberikan, gejala penyakit jantung koroner yang disebutkan adalah:

*   **Nyeri/rasa tertekan berat/tidak nyaman di dada.**
*   **Nyeri/tidak nyaman di dada dirasakan di dada bagian tengah/dada kiri depan/menjalar ke lengan kiri.**
*   **Nyeri/tidak nyaman di dada dirasakan ketika mendaki/naik tangga/berjalan tergesa-gesa.**
*   **Nyeri/tidak nyaman di dada hilang ketika menghentikan aktivitas/istirahat.**